# Install onnx and onnxruntime

In [ ]:
! pip install onnx onnxruntime

# Perform conversion

In [ ]:
import torch
import torchvision

import warnings
warnings.filterwarnings("ignore")

Use pretrained resnet50 model

In [ ]:
model = torchvision.models.resnet50(pretrained=True).cuda().eval()

Define dummy input

In [ ]:
data = torch.randn((1, 3, 224, 224)).cuda()

In [ ]:
output = model(data)
output_np = output.detach().cpu().numpy()

Export model to onnx

In [ ]:
model_path = '/content/drive/MyDrive/JetBot/Day 4/Examples/resnet50.onnx'
# model_path = 'resnet50.onnx' # Use this on Jetbot

torch.onnx.export(model, data, model_path)

# Perform Inference

In [ ]:
import onnxruntime
import numpy as np

Load model

In [ ]:
session = onnxruntime.InferenceSession(model_path, None)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(input_name, output_name)

Prepare inference data

In [ ]:
data_np = data.detach().cpu().numpy()

Run Inference

In [ ]:
result = session.run([], {input_name: data_np})

# Output of session is in list. Convert back to numpy for process
result_orig = np.array(result).squeeze()

Find the maximum error

In [ ]:
max_error = np.amax(np.abs(output_np - result_orig))
print('max error: %f' %max_error)

# Quantize onnx model

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
model_path_quant = '/content/drive/MyDrive/JetBot/Day 4/Examples/resnet50_quant.onnx'
# model_path_quant = 'resnet50_quant.onnx' # Use this on Jetbot

quantized_model = quantize_dynamic(model_path, model_path_quant, weight_type=QuantType.QUInt8)

In [ ]:
session = onnxruntime.InferenceSession(model_path_quant, None)

input_name = session.get_inputs()[0].name
print(input_name)

In [ ]:
result = session.run([], {input_name: data_np})

# Output of session is in list. Convert back to numpy for process
result_quant = np.array(result).squeeze()

Find the maximum error compared to pytorch model

In [ ]:
max_error = np.amax(np.abs(output_np - result))
print('max error: %f' %max_error)

Find the maximum error compared to original onnx model

In [ ]:
max_error = np.amax(np.abs(result_orig - result_quant))
print('max error: %f' %max_error)